# 加载`MNIST`数据集

In [3]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='D:\\code\workspace-self\\tensorflow-pg\\data\\mnist\\mnist.npz')
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


Using TensorFlow backend.


(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


# 数据处理：规范化
- `channels_last`: 表示图片格式为 `(batch, height, width, channels)`, 否则为`(batch, channels, height, width)`
- 可以通过`~/.keras/keras.json`文件中的 `channels_last`参数查看


In [5]:
from keras import backend as K

img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
    

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [6]:
# 将数据类型转换为`float32`
# 将数据类型转化为float32
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')

# 归一化
X_train /= 255
X_test /= 255


# 预测值进行编码`one-hot`


In [7]:
from keras.utils import np_utils

n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)
Y_test = np_utils.to_categorical(y_test, n_classes)


Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


# 使用`Keras sequential model`定义`MNIST CNN`网络


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D


In [9]:
model = Sequential()
## Extraction
# 第一层卷积层，32个3*3的卷积核，激活函数relu
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# 第二次卷积，64个3*3的卷积核，激活函数relu
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# 最大池化层，池化窗口 2*2
model.add(MaxPool2D(pool_size=(2, 2)))
# Dropout
model.add(Dropout(0.25))
# 摊平
model.add(Flatten())

## Classification
# 全连接层
model.add(Dense(128, activation='relu'))
# Dropout
model.add(Dropout(0.5))
# 使用 softmax 分类
model.add(Dense(n_classes, activation='softmax'))



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# 查看`MNIST CNN`模型网络结构


In [10]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [11]:
for layer in model.layers:
    print(layer.get_output_at(0).get_shape().as_list())
    

[None, 26, 26, 32]
[None, 24, 24, 64]
[None, 12, 12, 64]
[None, 12, 12, 64]
[None, None]
[None, 128]
[None, 128]
[None, 10]


# 编译模型


In [12]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


# 训练模型，将指标保存到`history`


In [13]:
history = model.fit(X_train,
                    Y_train,
                    batch_size=128,
                    epochs=5,
                    verbose=2,
                    validation_data=(X_test, Y_test))


Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
 - 95s - loss: 0.2317 - acc: 0.9308 - val_loss: 0.0508 - val_acc: 0.9836
Epoch 2/5
 - 91s - loss: 0.0797 - acc: 0.9758 - val_loss: 0.0401 - val_acc: 0.9858
Epoch 3/5
 - 90s - loss: 0.0603 - acc: 0.9815 - val_loss: 0.0324 - val_acc: 0.9895
Epoch 4/5
 - 90s - loss: 0.0489 - acc: 0.9848 - val_loss: 0.0334 - val_acc: 0.9888
Epoch 5/5
 - 90s - loss: 0.0413 - acc: 0.9876 - val_loss: 0.0307 - val_acc: 0.9901
